In [37]:
import numpy as np
from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 데이터셋 로드
import pandas as pd

df = pd.read_json('News_Category_Dataset_v3_balanced.json', lines=True)
display(df)



,link,headline,category,short_description,authors,date
0,https://www.huffpost.com/entry/covid-boosters-...,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...,"Carla K. Johnson, AP",2022-09-23
1,https://www.huffpost.com/entry/american-airlin...,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...,Mary Papenfuss,2022-09-23
2,https://www.huffpost.com/entry/funniest-tweets...,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha...",Elyse Wanshel,2022-09-23
3,https://www.huffpost.com/entry/funniest-parent...,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to...",Caroline Bologna,2022-09-23
4,https://www.huffpost.com/entry/amy-cooper-lose...,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...,Nina Golgowski,2022-09-22
...,...,...,...,...,...,...
209522,https://www.huffingtonpost.com/entry/rim-ceo-t...,RIM CEO Thorsten Heins' 'Significant' Plans Fo...,SCIENCE & TECH,Verizon Wireless and AT&T are already promotin...,"Reuters, Reuters",2012-01-28
209523,https://www.huffingtonpost.com/entry/maria-sha...,Maria Sharapova Stunned By Victoria Azarenka I...,SPORTS,"Afterward, Azarenka, more effusive with the pr...",,2012-01-28
209524,https://www.huffingtonpost.com/entry/super-bow...,"Giants Over Patriots, Jets Over Colts Among M...",SPORTS,"Leading up to Super Bowl XLVI, the most talked...",,2012-01-28
209525,https://www.huffingtonpost.com/entry/aldon-smi...,Aldon Smith Arrested: 49ers Linebacker Busted ...,SPORTS,CORRECTION: An earlier version of this story i...,,2012-01-28


In [38]:
from sklearn.model_selection import train_test_split

In [39]:
from tqdm import tqdm

docs = []
for i in tqdm(range(len(df))):{
    docs.append(df.iloc[i]['headline'])
}

100%|██████████| 209527/209527 [00:16<00:00, 12840.21it/s]


In [40]:
from gensim.models import Word2Vec
from sklearn.linear_model import LogisticRegression

In [41]:
labels = []
for i in tqdm(range(len(df))):{
    labels.append(df.iloc[i]['category'])
} 

100%|██████████| 209527/209527 [00:16<00:00, 12711.36it/s]


In [42]:
    word2vec_model = Word2Vec()

    # 각 문서를 단어로 분리하여 단어 사전 생성
    vocab = defaultdict(lambda: len(vocab))
    for doc in docs:
        for word in doc.split():
            vocab[word]

    # 단어 임베딩 벡터 생성
    embedding_dim = 100
    embeddings = np.random.uniform(low=-1, high=1, size=(len(vocab), embedding_dim))

    # 각 단어에 대한 임베딩 벡터 할당
    for word, idx in vocab.items():
        if word in word2vec_model.wv.index_to_key:
            embeddings[idx] = word2vec_model[word]

    # 각 문서의 임베딩 벡터 생성
    doc_embeddings = []
    for doc in docs:
        doc_emb = np.zeros(embedding_dim)
        for word in doc.split():
            doc_emb += embeddings[vocab[word]]
        doc_embeddings.append(doc_emb / len(doc.split()))

    # 데이터셋 분리
    X_train, X_test, y_train, y_test = train_test_split(doc_embeddings, labels, test_size=0.2, random_state=4321)



/tmp/ipykernel_643911/3323261720.py:24: RuntimeWarning: invalid value encountered in divide
  doc_embeddings.append(doc_emb / len(doc.split()))


In [43]:
# 분류 알고리즘으로 문서 분류
clf = LogisticRegression()
X_train_pre =  np.nan_to_num(X_train)
y_train_pre = np.nan_to_num(y_train)
clf.fit(X_train_pre, y_train_pre)
X_test_pre = np.nan_to_num(X_test)
y_pred = clf.predict(X_test_pre)

# 정확도 출력
print('Accuracy:', accuracy_score(y_test, y_pred))

Accuracy: 0.2395599675464134
